# Statistical Model Languages

Tarea 3 - Luis Eduardo Robles Jimenez

## Language Model and Evaluation

In [1]:
import nltk
import numpy as np
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split, ParameterGrid

### 1. Preprocessing

In [2]:
tokens = {'begin': '<s>', 'end': '</s>', 'unknown': '<unk>', 'separator': '<sep>'}

In [3]:
def _create_vocabulary(corpus, size):                           # Returns words sorted by frequency
    words, tokenizer, corpusByWords = [], TweetTokenizer(), []
    for doc in corpus:
        tokens = tokenizer.tokenize(doc)
        words += tokens
        corpusByWords += [tokens]
    count = nltk.FreqDist(words)
    count = sorted([(count[key], key) for key in count])[::-1]
    if size != -1: count = count[:size]
    return [word for _, word in count], corpusByWords

def load_corpus(corpus_select = "tweets", vocabSize = 100):
    corpus = []
    path_corpus = "../../data/agresividad/mex_train.txt"

    with open(path_corpus, "r") as f_corpus:
        for tuit in f_corpus:
            corpus += [tuit[:-1]]
    
    vocab, tokenized = _create_vocabulary(corpus, vocabSize)
    corpus = []
    for doc in tokenized:
        tweet = []
        tweet.append(tokens['begin'])
        for word in doc: 
            tweet.append(tokens['unknown'] if word not in vocab else word.lower())
        tweet.append(tokens['end'])
        corpus.append(tweet)
    return corpus

In [4]:
corpus = load_corpus(vocabSize = -100)

#### Comment

### 2. Models Training

# TODO: FIX LAPLACE SMOOTHING

# TODO: FIX PERPLEXITY

In [5]:
class LanguageModel:
    def __init__(self, corpus = None):
        self.corpus = corpus
        self.nGrams, self.vocab = dict(), set()
        for line in corpus:
            self.vocab.update(line)
            for gram in self.getNGrams(line):
                gram = self.toString(gram)
                if not gram in self.nGrams: self.nGrams[gram] = 0
                self.nGrams[gram] += 1
        self.vocab = list(self.vocab)

    def toString(self, gramList):
        gram = ""
        for i, g in enumerate(gramList):
            if i: gram += tokens['separator']
            gram += g
        return gram

    def P(self, *words):
        # Laplace smoothing
        assert len(words) == self.gramLen, "n-gram doesn't match the expected length"
        words = [(w if w in self.vocab else tokens['unknown']) for w in words]
        gram = self.toString(words)
        return self._Laplace(gram)

    def _Laplace(self, gram):
        count = 0
        if gram in self.nGrams: count = self.nGrams[gram]
        ctx = gram.split(tokens['separator'])[: -1]
        ctxCount = 0
        for gram in self.nGrams:
            if ctx == gram.split(tokens['separator'])[: -1]:
                ctxCount += self.nGrams[gram]
        return (count + 1) / (ctxCount + len(self.vocab))

    def getNGrams(self, line):
        return [line[start: start + self.gramLen] for start in range(len(line) - self.gramLen + 1)]

    def getProbs(self, sentence):
        # Naive approach to see if it's a list of lists
        if isinstance(sentence[0], list):
            sentence = sum(sentence, [])
        prob = 0
        for gram in self.getNGrams(sentence):
            prob += np.log(self.P(*gram))
        return np.exp(prob)

    # Include the <s> and </s> tokens, but don't count </s> - (Page 8, Dan Jurafsky on Language Models)
    def perplexity(self, sentence):
        # Naive approach to see if it's a list of lists
        if isinstance(sentence[0], list):
            sentence = sum(sentence, [])
        return self.getProbs(sentence) ** (-1 / (len(sentence) - sentence.count(tokens['begin'])) )

    def tweet(self, length = 50):
        tweet = [tokens['begin'] for _ in range(self.gramLen - 1)]
        for _ in range(length):
            ctx = tweet[-self.gramLen + 1:] if self.gramLen > 1 else []
            probs = []
            for i, w in enumerate(self.vocab):
                w = ctx + [w]
                p = self.P(*w)
                probs.append(p)
            tweet += [np.random.choice(self.vocab, p = probs / np.sum(probs))]
        for t in tweet: print(t + " ", end = "")
        return tweet

    def test(self):
        # Hypothesis: The sum of probabilities for a model is: vocabSize ^ (gramLength - 1)
        raise NotImplementedError()

# TODO FINISH INTERPOLATED MODEL

In [6]:
class Unigram(LanguageModel):
    def __init__(self, corpus = None):
        self.gramLen = 1
        super().__init__(corpus)

    def test(self):
        p = 0
        for w in self.vocab: p += self.P(w)
        assert np.round(p, decimals = 3) == 1, "Probs don't sum up the expected value"

class Bigram(LanguageModel):
    def __init__(self, corpus = None):
        self.gramLen = 2
        super().__init__(corpus)      

    def test(self):
        for w1 in self.vocab:
            p = 0
            for w2 in self.vocab:
                p += self.P(w1, w2)
            assert np.round(p, decimals = 3) == 1, "Probs don't sum up the expected value"

class Trigram(LanguageModel):
    def __init__(self, corpus = None):
        self.gramLen = 3
        super().__init__(corpus)    

    def test(self):
        for w1 in self.vocab:
            for w2 in self.vocab:
                p = 0   
                for w3 in self.vocab:
                    p += self.P(w1, w2, w3)
                assert np.round(p, decimals = 3) == 1, "Probs don't sum up the expected value"

class N_Gram(LanguageModel):
    def __init__(self, gramLen, corpus = None):
        self.gramLen = gramLen
        super().__init__(corpus)    

class Interpolated(LanguageModel):
    def __init__(self, corpus = None, models = None, lambdas = None):
        #super().__init__(corpus)   
        self.lambdas = np.array(lambdas)
        self.models = models
        assert len(models) == self.lambdas.shape[1], "The number of models doesn't match the number of lambdas"

    # Currently refactoring to deprecate
    def Interpolate(models, set, lambdas):
        indProbs = np.zeros((len(models)))
        lambdas = np.array(lambdas)
        for case in set:
            for m, model in enumerate(models):
                indProbs[m] += model.getProbs(case)
        indProbs /= len(set)
        probs = np.dot(indProbs, lambdas.T)
        return probs
    
    def _getProbs(self, sentence):
        # Gets the probability of each model
        indProbs = np.zeros((len(self.models)))
        for m, model in enumerate(self.models):
            indProbs[m] += model.getProbs(sentence)
        indProbs /= len(set)
    
    def dev(self, set):
        #for case in set:
        probs = np.dot(indProbs, self.lambdas.T)
        return probs

In [7]:
uni = Unigram(corpus)
#uni.test()

# Very frequent
print(uni.P("que"))
print(uni.P(tokens['begin']))

# Doesn't exist
print(uni.P("otorrinolaringologo"))

0.026867164736052336
0.044024358292378905
0.0008018864179495526


In [8]:
bi = Bigram(corpus)
#bi.test()

# Very frequent 
print(bi.P('.', tokens['end']))
print(bi.P("es", "que"))

# Doesn't exist
print(bi.P(tokens['begin'], tokens['end']))

0.08451743864181584
0.005644599303135889
5.255689283649551e-05


In [9]:
tri = Trigram(corpus)
#tri.test()

# Very frequent
print(tri.P('!', '!', '!'))
print(tri.P('es', 'que', 'no'))

# Doesn't exist
print(tri.P('Luis', 'Eduardo', 'Robles'))


0.014992184169390365
0.0006635700066357001
7.415647015202076e-05


#### Comment

### 3. Interpolated Model

In [24]:
c_train, c_test = train_test_split(corpus, test_size = 0.2, train_size = 0.8)
c_test, c_val = train_test_split(c_test, test_size = 0.5, train_size = 0.5)
print(f'Lengths of stratified sets:\n\tTrain: {len(c_train)}\n\tTest: {len(c_test)}\n\tValidation: {len(c_val)}')

Lengths of stratified sets:
	Train: 4435
	Test: 554
	Validation: 555


# TODO - Not needed but strange behavior

In [25]:
uni, bi, tri = Unigram(c_train), Bigram(c_train), Trigram(c_train)
pU, pB, pT = uni.perplexity(c_val), bi.perplexity(c_val), tri.perplexity(c_val)
print(pU, pB, pT)

# TODO 

In [ ]:
params = [(1/3, 1/3, 1/3), (0.4, 0.4, 0.2), (0.2, 0.4, 0.4), (0.5, 0.4, 0.1), (0.1, 0.4, 0.5), (0.9, 0.05, 0.05)]

In [ ]:
# This is wrong
'''
pps = Interpolated.Interpolate([uni, bi, tri], c_test, params) ** (-1 / len(c_test))
ordered = np.argsort(pps)
print("Params ordered by perplexity")
for p in ordered: print(f"\t{np.round(params[p], decimals = 1)} with a value of {pps[p]}")
'''

In [ ]:
'''
pps = Interpolated(models = [uni, bi, tri], lambdas = params).dev(c_test) ** (-1 / len(c_test))
ordered = np.argsort(pps)
print("Params ordered by perplexity")
for p in ordered: print(f"\t{np.round(params[p], decimals = 1)} with a value of {pps[p]}")
'''

#### Comment

## Text Generation

### 1. Tweet Functionality

In [10]:
uni.tweet(length = 10)

con que premiere también tienes modos gastadora chingaban evento tú 

['con',
 'que',
 'premiere',
 'también',
 'tienes',
 'modos',
 'gastadora',
 'chingaban',
 'evento',
 'tú']

In [11]:
bi.tweet(length = 10)

KeyboardInterrupt: 

In [ ]:
tri.tweet(length = 10)

<s> <s> / ☹ dos ✊ 🔥 muerto antes entiendo 😘 tú 

['<s>',
 '<s>',
 '/',
 '☹',
 'dos',
 '✊',
 '🔥',
 'muerto',
 'antes',
 'entiendo',
 '😘',
 'tú']

#### Comment

### 2. AMLO model

#### Comment

### 3. Evaluation with custom phrases

#### Comment

### 4. More evaluation 

#### Comment

## El ahorcado

### 1. Norvig's Hangman

#### Comment

### 2. Follow-up

#### Comment